Instalación

In [75]:
!pip install -q streamlit
!npm install localtunnel

Se crea el archivo con las lineas del EDA combinadas para mostrarlas en streamlit

In [76]:
%%writefile app.py
import streamlit as st
import pandas as pd

df_customers = pd.read_csv("https://raw.githubusercontent.com/Astroprogramm/PF-E-commerce-Olist/master/Datasets/olist_customers_dataset.csv")
df_order_items = pd.read_csv("https://raw.githubusercontent.com/Astroprogramm/PF-E-commerce-Olist/master/Datasets/olist_order_items_dataset.csv")
df_order_payments = pd.read_csv("https://raw.githubusercontent.com/Astroprogramm/PF-E-commerce-Olist/master/Datasets/olist_order_payments_dataset.csv")
df_order_reviews = pd.read_csv("https://raw.githubusercontent.com/Astroprogramm/PF-E-commerce-Olist/master/Datasets/olist_order_reviews_dataset.csv")
df_olist_orders = pd.read_csv("https://raw.githubusercontent.com/Astroprogramm/PF-E-commerce-Olist/master/Datasets/olist_orders_dataset.csv")
df_olist_products = pd.read_csv("https://raw.githubusercontent.com/Astroprogramm/PF-E-commerce-Olist/master/Datasets/olist_products_dataset.csv")

st.title('Dashboard KPIs olist')

# PRIMER KPI
st.header('1er KPI: Porcentaje de aumento o disminución en las ventas mensuales y trimestrales')
st.subheader('Variación mensual anual')

df_order_payments_data = df_order_payments[['order_id', 'payment_value']] 
df_olist_orders_data = df_olist_orders[['order_id', 'order_purchase_timestamp']]
inner_join_df= pd.merge(df_order_payments_data, df_olist_orders_data, on='order_id', how='inner')
inner_join_df['order_purchase_timestamp'] = pd.to_datetime(inner_join_df.order_purchase_timestamp, format='%Y-%m-%d')

option = st.selectbox(
    'Seleccione el año de la Variación mensual: ',
    (inner_join_df.order_purchase_timestamp.dt.year.unique()))
st.write('Seleccionó:', option)

inner_join_df = inner_join_df[inner_join_df['order_purchase_timestamp'].dt.year == int(option)]
st.bar_chart(inner_join_df.groupby(inner_join_df.order_purchase_timestamp.dt.month)['payment_value'].mean())

st.subheader('Variación trimestral anual')
df_order_payments_data = df_order_payments[['order_id', 'payment_value']] 
df_olist_orders_data = df_olist_orders[['order_id', 'order_purchase_timestamp']]

inner_join_df= pd.merge(df_order_payments_data, df_olist_orders_data, on='order_id', how='inner')
inner_join_df['order_purchase_timestamp'] = pd.to_datetime(inner_join_df.order_purchase_timestamp, format='%Y-%m-%d')
inner_join_df.groupby(inner_join_df.order_purchase_timestamp.dt.month)['payment_value'].mean()

option_trimestral = st.selectbox(
    'Seleccione el año de la variación trimestral: ',
    inner_join_df.order_purchase_timestamp.dt.year.unique())
st.write('Seleccionó:', option_trimestral)

inner_join_df = inner_join_df[inner_join_df['order_purchase_timestamp'].dt.year == int(option_trimestral)]

orders_by_month = len(inner_join_df)
orders_by_first_trimester = inner_join_df[inner_join_df.order_purchase_timestamp.dt.month <= 3 ]
orders_by_second_trimester = inner_join_df[(3 < inner_join_df.order_purchase_timestamp.dt.month) & (inner_join_df.order_purchase_timestamp.dt.month<=6)]
orders_by_third_trimester = inner_join_df[(6 < inner_join_df.order_purchase_timestamp.dt.month) & (inner_join_df.order_purchase_timestamp.dt.month<=9)]
orders_by_fourth_trimester = inner_join_df[(9 < inner_join_df.order_purchase_timestamp.dt.month) & (inner_join_df.order_purchase_timestamp.dt.month<=12)]

orders_by_first_trimester = inner_join_df.groupby(orders_by_first_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']
orders_by_second_trimester = inner_join_df.groupby(orders_by_second_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']
orders_by_third_trimester = inner_join_df.groupby(orders_by_third_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']
orders_by_fourth_trimester = inner_join_df.groupby(orders_by_fourth_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']

st.metric(label="Primer trimestre", value=((orders_by_first_trimester/orders_by_month)*100))
st.metric(label="Segundo trimestre", value=((orders_by_second_trimester/orders_by_month)*100))
st.metric(label="Segundo trimestre", value=((orders_by_third_trimester/orders_by_month)*100))
st.metric(label="Segundo trimestre", value=((orders_by_fourth_trimester/orders_by_month)*100))

# SEGUNDO KPI
st.header('2do KPI: Estados de Brazil en los que más se vende y menos se vende')
inner_join_df= pd.merge(df_customers, df_olist_orders, on='customer_id', how='inner')
orders_by_city = inner_join_df.groupby(inner_join_df.customer_state).nunique()
st.subheader('Estados con menos ventas')

estados = st.slider('Cuantos estados desea ver?', 3, 10, 3)
st.write("Visualizando ", estados, " estados.")

st.subheader('Estados con menos ventas')
listado_estados_menores = (orders_by_city.sort_values(by=['order_delivered_customer_date'])['order_delivered_customer_date'].head(estados))
st.table(listado_estados_menores)

st.subheader('Estados con mayores ventas')
listado_estados_mayores = (orders_by_city.sort_values(by=['order_delivered_customer_date'], ascending=False)['order_delivered_customer_date'].head(estados))
st.table(listado_estados_mayores)

# TERCER KPI
st.header('3er KPI: Frecuencia de compra mensual y trimestral.')
st.subheader('Porcentaje de frecuencia de compra mensual')
df_orders = df_olist_orders[['order_id']]
df_customer = df_olist_orders[['customer_id', 'order_purchase_timestamp']]
df_customer['order_purchase_timestamp'] = pd.to_datetime(df_customer.order_purchase_timestamp, format='%Y-%m-%d')
orders_by_month = len(df_orders)

option = st.selectbox(
    'Seleccione el año de la variación mensual: ',
    (df_customer.order_purchase_timestamp.dt.year.unique()))
st.write('Seleccionó:', option)

df_customer = df_customer[df_customer['order_purchase_timestamp'].dt.year == int(option)]

customers_by_month = df_customer.groupby(df_customer.order_purchase_timestamp.dt.month).nunique()
st.table((customers_by_month/orders_by_month)*100)

st.subheader('Porcentaje de frecuencia de compra trimestral')
df_customer = df_olist_orders[['customer_id', 'order_purchase_timestamp']]
df_customer['order_purchase_timestamp'] = pd.to_datetime(df_customer.order_purchase_timestamp, format='%Y-%m-%d')


option = st.selectbox(
    'Seleccione el año para calcular la variación trimestral: ',
    (df_customer.order_purchase_timestamp.dt.year.unique()))
st.write('Seleccionó:', option)

df_customer = df_customer[df_customer['order_purchase_timestamp'].dt.year == int(option)]

orders_by_month = len(df_customer)
customers_by_first_trimester = df_customer[df_customer.order_purchase_timestamp.dt.month <= 3 ]
customers_by_second_trimester = df_customer[(3 < df_customer.order_purchase_timestamp.dt.month) & (df_customer.order_purchase_timestamp.dt.month<=6)]
customers_by_third_trimester = df_customer[(6 < df_customer.order_purchase_timestamp.dt.month) & (df_customer.order_purchase_timestamp.dt.month<=9)]
customers_by_fourth_trimester = df_customer[(9 < df_customer.order_purchase_timestamp.dt.month) & (df_customer.order_purchase_timestamp.dt.month<=12)]

customers_by_first_trimester = df_customer.groupby(customers_by_first_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']
customers_by_second_trimester = df_customer.groupby(customers_by_second_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']
customers_by_third_trimester = df_customer.groupby(customers_by_third_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']
customers_by_fourth_trimester = df_customer.groupby(customers_by_fourth_trimester.order_purchase_timestamp.dt.month).nunique().mean()['order_purchase_timestamp']

st.metric(label="Primer trimestre", value=((customers_by_first_trimester/orders_by_month)*100))
st.metric(label="Segundo trimestre", value=((customers_by_second_trimester/orders_by_month)*100))
st.metric(label="Segundo trimestre", value=((customers_by_third_trimester/orders_by_month)*100))
st.metric(label="Segundo trimestre", value=((customers_by_fourth_trimester/orders_by_month)*100))

st.caption('Se compra más productos en el mes de agosto y en el segundo trimestre del año')

# CUARTO KPI
st.header('4to KPI: Net promoter score (NPS). Objetivo: Aumentar NPS 8%. Puntuación neta positivas')

calificacion = st.slider(
    'Seleccione un rango de calificaciones',
    0.0, 5.0, (4.0, 5.0))
st.write('Seleccionó:', calificacion[0], calificacion[1])

st.subheader('Puntuación positiva')
inner_join_df= pd.merge(df_order_reviews, df_order_items, on='order_id', how='inner')
sellers_positive_reviews = inner_join_df.groupby(['seller_id'])['review_score'].mean().to_frame()
sellers_positive_reviews = sellers_positive_reviews[(sellers_positive_reviews['review_score'] >= calificacion[0]) & (sellers_positive_reviews['review_score'] <= calificacion[1])]
st.table(sellers_positive_reviews.head(20))

# QUINTO KPI
st.header('5to KPI: Ticket promedio mensual y trimestral. Objetivo aumentar el ticket promedio mensual un 10%')
inner_join_df= pd.merge(df_olist_products, df_order_items, on='product_id', how='inner')
inner_join_df = inner_join_df.groupby(['product_category_name'])['price'].mean()

categorias = st.slider('Cuantos categorias desea ver?', 3, 10, 3)
st.write("Visualizando ", categorias, " categorias.")

st.subheader('Productos con mayor precio por categoría')
st.table(inner_join_df.sort_values(ascending=False).head(categorias))
st.subheader('Productos con menor precio por categoría')
st.table(inner_join_df.sort_values().head(categorias))

Overwriting app.py


Ejecutar streamlit

In [77]:
!streamlit run /content/app.py &>/content/logs.txt &

Exponer el puerto

In [78]:
!npx localtunnel --port 8501

npx: installed 22 in 2.954s
your url is: https://mighty-showers-drum-35-226-141-6.loca.lt
^C
